In [27]:
from bs4 import BeautifulSoup
from selenium import webdriver
from splinter import Browser
import urllib.parse
import pandas as pd
import pymongo
import requests
import os

# Nasa Mars News

In [28]:
news_url = "https://mars.nasa.gov/news/"
text = requests.get(news_url).text
news_file_name = "news-html-requests.txt"
news_file = "news-selenium-.txt"

In [29]:
with open(news_file_name, "w+") as f:
    f.write(text)

In [30]:
driver = webdriver.Firefox()
driver.implicitly_wait(40)
driver.get(news_url)
news_html = driver.page_source
driver.close()

In [31]:
with open(news_file, "w+") as f:
    f.write(news_html)

In [32]:
soup = BeautifulSoup(news_html, "html.parser")

In [33]:
news_article = soup.find("div", class_="list_text")
news_title = news_article.find("div", class_="content_title").text
news_p = news_article.find("div", class_="article_teaser_body").text

In [34]:
print(news_title)
print(news_p)

# JPL Mars Space Images

In [35]:
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
image_text = requests.get(image_url).text
soup = BeautifulSoup(image_text, "html.parser")

In [36]:
image = soup.find("div", class_="carousel_container")
for a in image.find_all("a", class_="button fancybox"):
    featured_image_url = urllib.parse.urljoin(
        "https://www.jpl.nasa.gov", a["data-fancybox-href"]
    )
    print(featured_image_url)

# Mars Weather

In [37]:
weather_url = "https://twitter.com/marswxreport?lang=en"
weather_file = "weather-html-requests.txt"
weather_text = requests.get(weather_url).text
soup = BeautifulSoup(weather_text, "html.parser")

In [38]:
with open(weather_file, "w+") as f:
    f.write(weather_text)

In [39]:
weather = soup.find("div", class_="js-tweet-text-container")

In [40]:
mars_weather = weather.find(
    "p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"
).text

In [41]:
mars_weather

# Mars Facts

In [42]:
facts_url = "https://space-facts.com/mars/"
facts_text = requests.get(facts_url).text
soup = BeautifulSoup(facts_text, "html.parser")

In [43]:
facts_table = soup.find("table")
table_rows = facts_table.find_all("tr")
l = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    l.append(row)
facts_df = pd.DataFrame(l)

In [44]:
facts_html = facts_df.to_html()

In [45]:
# facts_table = soup.find_all("table")[0] 
# facts_df = pd.read_html(str(table))

# Mars Hemispheres

In [46]:
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
driver = webdriver.Firefox()
driver.get(hemispheres_url)
driver.implicitly_wait(15)
hemispheres_html = driver.page_source
driver.close()

In [47]:
soup = BeautifulSoup(hemispheres_html, "html.parser")

In [48]:
items = soup.find_all("div", class_="item")

hemisphere_image_urls = []

for i in items:
    title = i.find("h3").text
    partial_url = i.find("a", class_="itemLink product-item")["href"]
    full_url = urllib.parse.urljoin("https://astrogeology.usgs.gov", partial_url)
    browser = Browser("firefox")
    browser.visit(full_url)
    partial_html = browser.html
    soup = BeautifulSoup(partial_html, "html.parser")
    img_url = (
        "https://astrogeology.usgs.gov" + soup.find("img", class_="wide-image")["src"]
    )
    hemisphere_image_urls.append({"title": title, "img_url": img_url})
    browser.quit()

In [49]:
hemisphere_image_urls